In [1]:
import pickle
import pandas as pd
import requests
import time
import json
from datetime import datetime, timedelta
import seaborn as sns
import time
import keras

Using TensorFlow backend.


In [2]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import EarlyStopping


# AQI to ppm ozone

In [3]:
def aqi2ppm(aqi):
    def new_scale(x, in_min, in_max, out_min,  out_max):
        return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min
    if (aqi<=50):
        ozone = new_scale(aqi, in_min=0, in_max=50, out_max=54, out_min=0)
    elif (aqi>50 and aqi<=100):
        ozone = new_scale(aqi, in_min=51, in_max=100, out_max=70, out_min=55)
    elif (aqi>100 and aqi<=150):
        ozone = new_scale(aqi, in_min=101, in_max=150, out_max=85, out_min=71)
    elif (aqi>150 and aqi<=200):
        ozone = new_scale(aqi, in_min=151, in_max=200, out_max=105, out_min=86) 
    elif (aqi>200 and aqi<=300):
        ozone = new_scale(aqi, in_min=200, in_max=300, out_max=200, out_min=106)
    elif (aqi>300 and aqi<=500):
        ozone = new_scale(aqi, in_min=301, in_max=500, out_max=600, out_min=201) 
    return (ozone/1000)

# Pull 7 days of Ozone and Weather

In [4]:
## AIR NOW fo OZONE
airnowkey = pickle.load(open('airnowAPIkey.p','rb'))

sfcurrenturl='http://www.airnowapi.org/aq/observation/zipCode/current/?format=text/csv&zipCode=94117&distance=25&API_KEY='+airnowkey
sf_ozone= pd.read_csv(sfcurrenturl)
sf_today= aqi2ppm(sf_ozone[sf_ozone['ParameterName']=='O3'].loc[:,'AQI'][0])


historical_ozone = [sf_today]
for day in range(1,7):
    year =str((datetime.today() -timedelta(days=day)).year)
    month= str((datetime.today() -timedelta(days=day)).month)
    day= str((datetime.today() -timedelta(days=day)).day)

    url='http://www.airnowapi.org/aq/observation/zipCode/historical/?format=text/csv&zipCode=94117&date='+year+'-'+month+'-'+day+'T00-0000&distance=25&API_KEY='+airnowkey

    raw=pd.read_csv(url)
    value=aqi2ppm(raw[raw['ParameterName']=='OZONE'].loc[:,'AQI'][0])
    historical_ozone.append(value)
historical_ozone.reverse()
historical_ozone.append(0)
historical_ozone.append(0)
historical_ozone.append(0)
historical_ozone.append(0)
historical_ozone.append(0)


#Dark SKY for weather

dskey = pickle.load(open("darkskyapi.p",'rb'))



day_length =[]
temp_high =[]
temp_low =[]
dew_point =[]
humidity =[]
pressure =[]
wind_speed =[]
wind_bearing =[]
cloud_cover =[]
uv_index =[]
date =[]

for i in range(-5,7):
    try:
        long = 122.4482
        lat = 37.7697
        day = datetime.today() -timedelta(days=i)
        day = int(time.mktime(day.timetuple()))
        url ='https://api.darksky.net/forecast/'+dskey+'/'+str(lat)+','+str(long)+','+str(day)
        weather= requests.get(url)
        day_length.append(weather.json()['daily']['data'][0]['sunsetTime'] - weather.json()['daily']['data'][0]['sunriseTime'])
        try:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureHigh'])
        except:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureMax'])
        try:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureLow'])
        except:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureMin'])
        dew_point.append(weather.json()['daily']['data'][0]['dewPoint'])
        humidity.append(weather.json()['daily']['data'][0]['humidity'])
        pressure.append(weather.json()['daily']['data'][0]['pressure'])
        wind_speed.append(weather.json()['daily']['data'][0]['windSpeed'])
        wind_bearing.append(weather.json()['daily']['data'][0]['windBearing'])
        cloud_cover.append(weather.json()['daily']['data'][0]['cloudCover'])
        uv_index.append(weather.json()['daily']['data'][0]['uvIndex'])
        date.append(pd.to_datetime(weather.json()['daily']['data'][0]['time'],unit='s'))
    except:
        print('broken at: ' +str(i))
day_length.reverse()
temp_high.reverse()
temp_low.reverse()
dew_point.reverse()
humidity.reverse()
pressure.reverse()
wind_speed.reverse()
wind_bearing.reverse()
cloud_cover.reverse()
uv_index.reverse()
date.reverse()

df= pd.DataFrame({
    'Ozone': historical_ozone,
    'day_length' : day_length,
    'temp_high' : temp_high,
    'temp_low' : temp_low,
    'dew_point' : dew_point,
    'humidity' : humidity,
    'pressure' : pressure,
    'wind_speed' : wind_speed,
    'wind_bearing' : wind_bearing,
    'cloud_cover' : cloud_cover,
    'uv_index' :uv_index
    
    
})
## load SF scaler
scaler = pickle.load(open('scaler_all.p','rb'))

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
## load SF prediction scaler
pred_scaler=pickle.load(open('pred_scaler.p','rb'))

# load dataset
values = df.values
# integer encode direction
# ensure all data is float
values = df.astype('float32')
# normalize features
scaler = pickle.load(open('scaler_all.p','rb'))
scaled = scaler.transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 5)
# drop columns we don't want to predict
five_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)'], axis =1).values
five_day = five_day.reshape((five_day.shape[0], 1, five_day.shape[1]))
five_day_lstm = keras.models.load_model('models/multi5day.h5')

sf_five = pred_scaler.inverse_transform(five_day_lstm.predict(five_day))[0][0]

four_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)'
                         ], axis =1).values

four_day = four_day.reshape((four_day.shape[0], 1, four_day.shape[1]))
four_day_lstm = keras.models.load_model('models/multi4day.h5')

sf_four = pred_scaler.inverse_transform(four_day_lstm.predict(four_day))[0][0]

three_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)',
                           'var2(t+3)','var3(t+3)','var4(t+3)','var5(t+3)','var6(t+3)',
                          'var7(t+3)','var8(t+3)','var9(t+3)','var10(t+3)','var11(t+3)'
                         ], axis =1).values

three_day = three_day.reshape((three_day.shape[0], 1, three_day.shape[1]))
three_day_lstm = keras.models.load_model('models/multi3day.h5')

sf_three = pred_scaler.inverse_transform(three_day_lstm.predict(three_day))[0][0]

two_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)',
                          'var2(t+3)','var3(t+3)','var4(t+3)','var5(t+3)','var6(t+3)',
                          'var7(t+3)','var8(t+3)','var9(t+3)','var10(t+3)','var11(t+3)',
                          'var2(t+2)','var3(t+2)','var4(t+2)','var5(t+2)','var6(t+2)',
                          'var7(t+2)','var8(t+2)','var9(t+2)','var10(t+2)','var11(t+2)'
                         ], axis =1).values

two_day = two_day.reshape((two_day.shape[0], 1, two_day.shape[1]))
two_day_lstm = keras.models.load_model('models/multi2day.h5')

sf_two = pred_scaler.inverse_transform(two_day_lstm.predict(two_day))[0][0]

one_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)',
                          'var2(t+3)','var3(t+3)','var4(t+3)','var5(t+3)','var6(t+3)',
                          'var7(t+3)','var8(t+3)','var9(t+3)','var10(t+3)','var11(t+3)',
                          'var2(t+2)','var3(t+2)','var4(t+2)','var5(t+2)','var6(t+2)',
                          'var7(t+2)','var8(t+2)','var9(t+2)','var10(t+2)','var11(t+2)',
                          'var2(t+1)','var3(t+1)','var4(t+1)','var5(t+1)','var6(t+1)',
                          'var7(t+1)','var8(t+1)','var9(t+1)','var10(t+1)','var11(t+1)'
                         ], axis =1).values

one_day = one_day.reshape((one_day.shape[0], 1, one_day.shape[1]))
one_day_lstm = keras.models.load_model('models/multi1day.h5')

sf_one = pred_scaler.inverse_transform(one_day_lstm.predict(one_day))[0][0]



In [5]:

airnowkey = pickle.load(open('airnowAPIkey.p','rb'))

lacurrenturl='http://www.airnowapi.org/aq/observation/zipCode/current/?format=text/csv&zipCode=91702&distance=25&API_KEY='+airnowkey
la_ozone= pd.read_csv(lacurrenturl)
la_today= aqi2ppm(la_ozone[la_ozone['ParameterName']=='O3'].loc[:,'AQI'][0])


historical_ozone = [la_today]
for day in range(1,7):
    year =str((datetime.today() -timedelta(days=day)).year)
    month= str((datetime.today() -timedelta(days=day)).month)
    day= str((datetime.today() -timedelta(days=day)).day)

    url='http://www.airnowapi.org/aq/observation/zipCode/historical/?format=text/csv&zipCode=91702&date='+year+'-'+month+'-'+day+'T00-0000&distance=25&API_KEY='+airnowkey

    raw=pd.read_csv(url)
    value=aqi2ppm(raw[raw['ParameterName']=='OZONE'].loc[:,'AQI'][0])
    historical_ozone.append(value)
historical_ozone.reverse()
historical_ozone.append(0)
historical_ozone.append(0)
historical_ozone.append(0)
historical_ozone.append(0)
historical_ozone.append(0)





#Dark SKY for weather

dskey = pickle.load(open("darkskyapi.p",'rb'))



day_length =[]
temp_high =[]
temp_low =[]
dew_point =[]
humidity =[]
pressure =[]
wind_speed =[]
wind_bearing =[]
cloud_cover =[]
uv_index =[]
date =[]

for i in range(-5,7):
    try:
        long = -117.92391
        lat = 34.1365
        day = datetime.today() -timedelta(days=i)
        day = int(time.mktime(day.timetuple()))
        url ='https://api.darksky.net/forecast/'+dskey+'/'+str(lat)+','+str(long)+','+str(day)
        weather= requests.get(url)
        day_length.append(weather.json()['daily']['data'][0]['sunsetTime'] - weather.json()['daily']['data'][0]['sunriseTime'])
        try:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureHigh'])
        except:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureMax'])
        try:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureLow'])
        except:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureMin'])
        dew_point.append(weather.json()['daily']['data'][0]['dewPoint'])
        humidity.append(weather.json()['daily']['data'][0]['humidity'])
        pressure.append(weather.json()['daily']['data'][0]['pressure'])
        wind_speed.append(weather.json()['daily']['data'][0]['windSpeed'])
        wind_bearing.append(weather.json()['daily']['data'][0]['windBearing'])
        cloud_cover.append(weather.json()['daily']['data'][0]['cloudCover'])
        uv_index.append(weather.json()['daily']['data'][0]['uvIndex'])
        date.append(pd.to_datetime(weather.json()['daily']['data'][0]['time'],unit='s'))
    except:
        print('broken at: ' +str(i))
day_length.reverse()
temp_high.reverse()
temp_low.reverse()
dew_point.reverse()
humidity.reverse()
pressure.reverse()
wind_speed.reverse()
wind_bearing.reverse()
cloud_cover.reverse()
uv_index.reverse()
date.reverse()

df= pd.DataFrame({
    'Ozone': historical_ozone,
    'day_length' : day_length,
    'temp_high' : temp_high,
    'temp_low' : temp_low,
    'dew_point' : dew_point,
    'humidity' : humidity,
    'pressure' : pressure,
    'wind_speed' : wind_speed,
    'wind_bearing' : wind_bearing,
    'cloud_cover' : cloud_cover,
    'uv_index' :uv_index
    
    
})
## load LA scaler
scaler = pickle.load(open('LA_all_scaled.p','rb'))

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
## load SF prediction scaler
pred_scaler=pickle.load(open('LA_pred_scaler.p','rb'))

# load dataset
values = df.values
# integer encode direction
# ensure all data is float
values = df.astype('float32')
# normalize features
scaler = pickle.load(open('scaler_all.p','rb'))
scaled = scaler.transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 5)
# drop columns we don't want to predict
five_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)'], axis =1).values
five_day = five_day.reshape((five_day.shape[0], 1, five_day.shape[1]))
five_day_lstm = keras.models.load_model('LAmodels/multi5day.h5')

la_five = pred_scaler.inverse_transform(five_day_lstm.predict(five_day))[0][0]

four_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)'
                         ], axis =1).values

four_day = four_day.reshape((four_day.shape[0], 1, four_day.shape[1]))
four_day_lstm = keras.models.load_model('LAmodels/multi4day.h5')

la_four = pred_scaler.inverse_transform(four_day_lstm.predict(four_day))[0][0]

three_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)',
                           'var2(t+3)','var3(t+3)','var4(t+3)','var5(t+3)','var6(t+3)',
                          'var7(t+3)','var8(t+3)','var9(t+3)','var10(t+3)','var11(t+3)'
                         ], axis =1).values

three_day = three_day.reshape((three_day.shape[0], 1, three_day.shape[1]))
three_day_lstm = keras.models.load_model('LAmodels/multi3day.h5')

la_three = pred_scaler.inverse_transform(three_day_lstm.predict(three_day))[0][0]

two_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)',
                          'var2(t+3)','var3(t+3)','var4(t+3)','var5(t+3)','var6(t+3)',
                          'var7(t+3)','var8(t+3)','var9(t+3)','var10(t+3)','var11(t+3)',
                          'var2(t+2)','var3(t+2)','var4(t+2)','var5(t+2)','var6(t+2)',
                          'var7(t+2)','var8(t+2)','var9(t+2)','var10(t+2)','var11(t+2)'
                         ], axis =1).values

two_day = two_day.reshape((two_day.shape[0], 1, two_day.shape[1]))
two_day_lstm = keras.models.load_model('LAmodels/multi2day.h5')

la_two = pred_scaler.inverse_transform(two_day_lstm.predict(two_day))[0][0]

one_day = reframed.drop(['var1(t)','var1(t+1)','var1(t+2)','var1(t+3)','var1(t+4)',
                          'var2(t+4)','var3(t+4)','var4(t+4)','var5(t+4)','var6(t+4)',
                          'var7(t+4)','var8(t+4)','var9(t+4)','var10(t+4)','var11(t+4)',
                          'var2(t+3)','var3(t+3)','var4(t+3)','var5(t+3)','var6(t+3)',
                          'var7(t+3)','var8(t+3)','var9(t+3)','var10(t+3)','var11(t+3)',
                          'var2(t+2)','var3(t+2)','var4(t+2)','var5(t+2)','var6(t+2)',
                          'var7(t+2)','var8(t+2)','var9(t+2)','var10(t+2)','var11(t+2)',
                          'var2(t+1)','var3(t+1)','var4(t+1)','var5(t+1)','var6(t+1)',
                          'var7(t+1)','var8(t+1)','var9(t+1)','var10(t+1)','var11(t+1)'
                         ], axis =1).values

one_day = one_day.reshape((one_day.shape[0], 1, one_day.shape[1]))
one_day_lstm = keras.models.load_model('LAmodels/multi1day.h5')

la_one = pred_scaler.inverse_transform(one_day_lstm.predict(one_day))[0][0]




In [6]:
la_forecast=[la_one,la_two,la_three,la_four,la_five]
sf_forecast=[sf_one,sf_two,sf_three,sf_four,sf_five]

dates = [date[-5],date[-4],date[-3],date[-2],date[-1]]




LAforecast = pd.DataFrame({
    "Date": dates,
    "Ozone(ppm)": la_forecast
})
SFforecast =  pd.DataFrame({
    "Date": dates,
    "Ozone(ppm)":sf_forecast
})

LAforecast['Date']=LAforecast['Date'].dt.strftime('%Y-%m-%d')

SFforecast['Date']=SFforecast['Date'].dt.strftime('%Y-%m-%d')

In [17]:
LAforecast = LAforecast.round({'Ozone(ppm)':3})

In [18]:
LAforecast

,Date,Ozone(ppm)
0,2019-06-19,0.079
1,2019-06-20,0.071
2,2019-06-21,0.060
3,2019-06-22,0.079
4,2019-06-23,0.089


In [19]:
SFforecast = SFforecast.round({'Ozone(ppm)':3})

In [20]:
SFforecast

,Date,Ozone(ppm)
0,2019-06-19,0.020
1,2019-06-20,0.022
2,2019-06-21,0.023
3,2019-06-22,0.022
4,2019-06-23,0.023
